In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import requests
from PIL import Image
from io import BytesIO
import numpy as np

# 데이터 불러오기
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'foodornot',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # 이진 분류 (음식, 음식이 아님)
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'foodornot',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # 이진 분류 (음식, 음식이 아님)
    subset='validation'
)

# 모델 아키텍처 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# 클래스 불균형을 위한 클래스별 샘플 수 계산
total_samples = train_generator.samples
class_counts = train_generator.classes
class_weights = {0: total_samples / (2 * np.bincount(class_counts)[0]),
                 1: total_samples / (2 * np.bincount(class_counts)[1])}

# 모델 컴파일 (클래스 가중치 추가)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 모델 학습 (클래스 가중치 적용)
history = model.fit(train_generator, validation_data=validation_generator, epochs=20, callbacks=[early_stopping], class_weight=class_weights)

# 새로운 이미지 다운로드
new_image_url = 'https://postfiles.pstatic.net/MjAyMzAyMjFfMjE4/MDAxNjc2OTU3Nzk0Nzk0.HaOn7wOIFAX0MllJBB4EKW14QfyKhCNcdgaUys56jf4g.iY39kPpvhbAuQ2YqUgid_rFioFgqanz4VA9rmH74-sIg.JPEG.qpwoei2017/IMG_3026.JPG?type=w966'
response = requests.get(new_image_url)
new_image = Image.open(BytesIO(response.content))

# 이미지 크기 조정 및 전처리
new_image = new_image.resize((224, 224))
new_image_array = img_to_array(new_image)
new_image_array = new_image_array / 255.0
new_image_array = tf.expand_dims(new_image_array, axis=0)

# 모델 예측
prediction = model.predict(new_image_array)
if prediction[0] < 0.5:
    print('food')
else:
    print('else')

Found 2323 images belonging to 2 classes.
Found 580 images belonging to 2 classes.
Epoch 1/20
73/73 [==============================] - 120s 2s/step - loss: 0.5417 - accuracy: 0.7568 - val_loss: 0.3704 - val_accuracy: 0.8483
Epoch 2/20
73/73 [==============================] - 122s 2s/step - loss: 0.3479 - accuracy: 0.8545 - val_loss: 0.2112 - val_accuracy: 0.9293
Epoch 3/20
73/73 [==============================] - 109s 1s/step - loss: 0.2519 - accuracy: 0.9001 - val_loss: 0.5017 - val_accuracy: 0.7690
Epoch 4/20
73/73 [==============================] - 127s 2s/step - loss: 0.2069 - accuracy: 0.9277 - val_loss: 0.1805 - val_accuracy: 0.9259
Epoch 5/20
73/73 [==============================] - 154s 2s/step - loss: 0.1138 - accuracy: 0.9608 - val_loss: 0.1358 - val_accuracy: 0.9483
Epoch 6/20
73/73 [==============================] - 138s 2s/step - loss: 0.0515 - accuracy: 0.9841 - val_loss: 0.0790 - val_accuracy: 0.9707
Epoch 7/20
73/73 [==============================] - 149s 2s/step - loss

In [15]:
import pickle

# 모델 저장
with open("food_classification_model.pkl", "wb") as f:
    pickle.dump(model, f)

In [18]:
import pickle

# 모델 불러오기
with open("food_classification_model.pkl", "rb") as f:
    model.predict = pickle.load(f)
    
    
# 새로운 이미지 다운로드
new_image_url = 'https://postfiles.pstatic.net/MjAyMzAyMjFfMjE4/MDAxNjc2OTU3Nzk0Nzk0.HaOn7wOIFAX0MllJBB4EKW14QfyKhCNcdgaUys56jf4g.iY39kPpvhbAuQ2YqUgid_rFioFgqanz4VA9rmH74-sIg.JPEG.qpwoei2017/IMG_3026.JPG?type=w966'
response = requests.get(new_image_url)
new_image = Image.open(BytesIO(response.content))

# 이미지 크기 조정 및 전처리
new_image = new_image.resize((224, 224))
new_image_array = img_to_array(new_image)
new_image_array = new_image_array / 255.0
new_image_array = tf.expand_dims(new_image_array, axis=0)

# 모델 예측
prediction = model.predict(new_image_array)
if prediction[0] < 0.5:
    print('food')
else:
    print('else')

food


In [19]:
# 새로운 이미지 예측
new_image_path = 'tf3.jpg'
new_image = tf.keras.preprocessing.image.load_img(new_image_path, target_size=(224, 224))
new_image = tf.keras.preprocessing.image.img_to_array(new_image)
new_image = new_image / 255.0
new_image = tf.expand_dims(new_image, axis=0)

prediction = model.predict(new_image)
if prediction[0] < 0.5:
    print('food')
else:
    print('else')

food


In [20]:
# 새로운 이미지 예측
new_image_path = 'de.jpg'
new_image = tf.keras.preprocessing.image.load_img(new_image_path, target_size=(224, 224))
new_image = tf.keras.preprocessing.image.img_to_array(new_image)
new_image = new_image / 255.0
new_image = tf.expand_dims(new_image, axis=0)

prediction = model.predict(new_image)
if prediction[0] < 0.5:
    print('food')
else:
    print('else')

else


In [21]:
# 새로운 이미지 다운로드
new_image_url = 'https://postfiles.pstatic.net/MjAyMzAyMjFfMjE4/MDAxNjc2OTU3Nzk0Nzk0.HaOn7wOIFAX0MllJBB4EKW14QfyKhCNcdgaUys56jf4g.iY39kPpvhbAuQ2YqUgid_rFioFgqanz4VA9rmH74-sIg.JPEG.qpwoei2017/IMG_3026.JPG?type=w966'
response = requests.get(new_image_url)
new_image = Image.open(BytesIO(response.content))

# 이미지 크기 조정 및 전처리
new_image = new_image.resize((224, 224))
new_image_array = img_to_array(new_image)
new_image_array = new_image_array / 255.0
new_image_array = tf.expand_dims(new_image_array, axis=0)

# 모델 예측
prediction = model.predict(new_image_array)
if prediction[0] < 0.5:
    print('food')
else:
    print('else')

food


In [5]:
# 새로운 이미지 다운로드
new_image_url = 'https://postfiles.pstatic.net/MjAyMzA0MDRfMjMz/MDAxNjgwNTM1MjQ5OTk4.aG_e0P5Y5Oh_KJXiJJsR-HuKwrjxSMyyxaPmVA2nBQAg.x425b7V3Ubj40kClwflILowIQLkZYPlmuSSNC5AX1gIg.JPEG.vnemfl22/1680535243708.jpg?type=w580'
response = requests.get(new_image_url)
new_image = Image.open(BytesIO(response.content))

# 이미지 크기 조정 및 전처리
new_image = new_image.resize((224, 224))
new_image_array = img_to_array(new_image)
new_image_array = new_image_array / 255.0
new_image_array = tf.expand_dims(new_image_array, axis=0)

# 모델 예측
prediction = model.predict(new_image_array)
if prediction[0] < 0.8:
    print('food')
else:
    print('else')

1/1 [==============================] - 0s 55ms/step
else


## 여기서부터 코드 정리 필요

In [16]:
 pip install PyMySQL

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pymysql
blogdb = pymysql.connect(host='blogdb.cm2yxwfja9ii.ap-northeast-2.rds.amazonaws.com',
                      user='admin',
                      password='blogdb!2',
                      database='blogdb',
                      charset='utf8',
                      port=3306)

cursor =blogdb.cursor(pymysql.cursors.DictCursor)

# 데이터불러오기
query = "SELECT p.url AS post_url, pds.url AS post_data_set_url, pds.images FROM post p JOIN post_data_set pds ON p.url = pds.url; "
cursor.execute(query)
results = cursor.fetchall()

KeyboardInterrupt: 

In [17]:
results

[{'post_url': 'https://blog.naver.com/baebaeea/223144029665',
  'post_data_set_url': 'https://blog.naver.com/baebaeea/223144029665',
  'images': 'https://postfiles.pstatic.net/MjAyMzA2MjlfMTAz/MDAxNjg4MDM4NTAzMDIy.mLGC8_XYGAF_XneDOMvHbVXhwgcK0pATNYq_rHEihecg.nbw3IGGWgiAwy-oLj768yTyqaXcWRyE2wcNdk1YH72Qg.JPEG.mings17/SE-A823DA16-42A2-4F00-9974-337383A7EFAF.jpg?type=w966\nhttps://postfiles.pstatic.net/MjAyMzA2MjlfMTI4/MDAxNjg4MDM4NTAzNzg5.8-46T2XvxbjVjGBQdRyJ1ysYXdjYos5ZyuZKhaBi1asg.LXVDuZzTQLERkucF1bTfOWY4LUWrrQvZGAl-A-6R8wgg.JPEG.mings17/SE-D37A10DF-8DE6-49E8-AEF9-E53BC4755B9E.jpg?type=w966\nhttps://postfiles.pstatic.net/MjAyMzA2MjlfMjg2/MDAxNjg4MDM4NTA0Mzg1.QCUbAhnZYh2O-qcpl7YN8UHpLwoa30J9VwTFacY1APYg.20gRFU9YhdMk5_1FYHSSjT5RnclQmLYUJBMaVgLRiK8g.JPEG.mings17/SE-99F0B38A-B8BE-4AC0-9E11-886183F0FE37.jpg?type=w966\nhttps://postfiles.pstatic.net/MjAyMzA2MjlfMjg2/MDAxNjg4MDM4NTA1MjEy.FpEOCDp5L89C2mr5upS8be3xWFBAstey4uQGuumIPXUg.gld0KIga21WDd2xrXK_8rOfZxdGpB_GSSN2WBbkpuWUg.JPEG.mings17/SE-40

In [ ]:
import pickle

# 모델 불러오기
with open("food_classification_model.pkl", "rb") as f:
    model.predict = pickle.load(f)
    
    
# 새로운 이미지 다운로드
new_image_url = 'https://postfiles.pstatic.net/MjAyMzAyMjFfMjE4/MDAxNjc2OTU3Nzk0Nzk0.HaOn7wOIFAX0MllJBB4EKW14QfyKhCNcdgaUys56jf4g.iY39kPpvhbAuQ2YqUgid_rFioFgqanz4VA9rmH74-sIg.JPEG.qpwoei2017/IMG_3026.JPG?type=w966'
response = requests.get(new_image_url)
new_image = Image.open(BytesIO(response.content))

# 이미지 크기 조정 및 전처리
new_image = new_image.resize((224, 224))
new_image_array = img_to_array(new_image)
new_image_array = new_image_array / 255.0
new_image_array = tf.expand_dims(new_image_array, axis=0)

# 모델 예측
prediction = model.predict(new_image_array)
if prediction[0] < 0.5:
    print('food')
else:
    print('else')

In [ ]:
# 결과 리스트(results)에서 이미지 주소들 가져오기
for result in results:
    image_urls = result['images'].split('\n')
    
    for image_url in image_urls:
        # 이미지 다운로드
        response = requests.get(image_url)
        new_image = Image.open(BytesIO(response.content))

        # 이미지 크기 조정 및 전처리
        new_image = new_image.resize((224, 224))
        new_image_array = img_to_array(new_image)
        new_image_array = new_image_array / 255.0
        new_image_array = tf.expand_dims(new_image_array, axis=0)

        # 모델 예측
        prediction = model.predict(new_image_array)
        
        if prediction[0] < 0.5:
            print('food')
        else:
            print('else')

In [12]:
for result in results:
    image_urls = result['images'].split('\n')
    selected_url = None  # 선택된 이미지 URL을 저장하는 변수
    
    for image_url in image_urls:
        response = requests.get(image_url)
        new_image = Image.open(BytesIO(response.content))
        new_image = new_image.resize((224, 224))
        new_image_array = img_to_array(new_image)
        new_image_array = new_image_array / 255.0
        new_image_array = tf.expand_dims(new_image_array, axis=0)
        
        try:
            prediction = model.predict(new_image_array)
            
            if prediction[0] < 0.5:
                print(f"Found 'food' image: {image_url}")
                selected_url = image_url
                break  # 작업 중단
        
        except Exception as e:
            print(f"Error occurred for image {image_url}: {e}")
            selected_url = image_urls[0]  # 에러 발생 시 첫 번째 URL 선택
            break
    
    # food로 분류되는 이미지가 없을 경우, 젤 위의 URL 선택
    if selected_url is None and len(image_urls) > 0:
        selected_url = image_urls[0]
        print(f"No 'food' images found. Selecting the top URL: {selected_url}")
    
    if selected_url:
        # 이미지 예측 결과가 'food'인 경우 해당 이미지 URL을 post 테이블의 food_img_url 컬럼에 업데이트
        update_query = f"UPDATE post SET food_img_url = '{selected_url}' WHERE url = '{result['post_url']}';"
        cursor.execute(update_query)
        blogdb.commit()

NameError: name 'results' is not defined